In [1]:
import requests
from PIL import Image
from googletrans import Translator
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv('translated_full.csv', sep=',')
df.head()

,index,object_id,name,description,group,img_name,img_path,translated
0,1363,19943072,Пластинка,"Всероссийское театральное общество ВТО, фабрик...",Прочие,20650863.jpg,dataset/train/19943072/20650863.jpg,"All -Russian Theater Society of the WTO, a fac..."
1,3147,9221257,Каталог. Первая Бельгïйская Художественная Выс...,В бумажной обложке с полихромной печатью титул...,Редкие книги,31817178.jpg,dataset/train/9221257/31817178.jpg,In the paper cover with the polychrome printin...
2,4864,43808403,Письмо к Новикову Ивану Алексеевичу.,С просьбой назначить представителя от Госиздат...,Документы,53905453.jpg,dataset/train/43808403/53905453.jpg,With a request to appoint a representative fro...
3,4527,19941202,Бинокль.,Бинокль. Производитель – CP GOERZ (оттиск штам...,Техника,20648209.jpg,dataset/train/19941202/20648209.jpg,Binoculars.Manufacturer - CP Goerz (print of t...
4,4771,21625781,Радиоприемник «Домбай-303»,"Черного цвета, серая металлическая вставка. в ...",Техника,22984135.jpg,dataset/train/21625781/22984135.jpg,"Black, gray metal insert.In the lower part of ..."


In [3]:
df["img_path"] = df["img_path"].apply(lambda x: x.replace("dataset", "E:\MachineLearningProjects\ml-practices\src\haccaton\data"))

In [4]:
translator = Translator()
def translate_text(text, dest_lang='ru', src='en'):
    translation = translator.translate(text, dest=dest_lang, src=src)
    return translation.text

In [5]:
len(df)

10280

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, df['group'], test_size=0.1, random_state=0)

In [7]:
my_dataset = []
for i in range(len(X_train)):
    my_dataset.append(f'{X_train.iloc[i]["img_path"]}!****!{X_train.iloc[i]["translated"]}')
print(my_dataset[0])

E:\MachineLearningProjects\ml-practices\src\haccaton\data/train/27501326/31231304.jpg!****!A round shape with a square hole in the center, on the front side - four hieroglyphs located crossly on the back - two side hieroglyphs.


In [8]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=np.array(Image.open(item.split("!****!")[0])), text="a photography of", padding="max_length", return_tensors="pt")
        labels = self.processor.tokenizer.encode(
            item.split("!****!")[1], max_length= 512, pad_to_max_length=True, return_tensors='pt'
        )
        encoding["labels"] = labels
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [9]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [10]:
train_dataset = ImageCaptioningDataset(my_dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)

In [11]:
my_dataset = []
for i in range(len(X_train)):
    my_dataset.append(f'{X_train.iloc[i]["img_path"]}!****!{X_train.iloc[i]["translated"]}')
print(my_dataset[0])

E:\MachineLearningProjects\ml-practices\src\haccaton\data/train/27501326/31231304.jpg!****!A round shape with a square hole in the center, on the front side - four hieroglyphs located crossly on the back - two side hieroglyphs.


In [12]:
import torch
from tqdm.auto import tqdm

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
count = 0
model.train()

for epoch in range(5):
    print("Epoch:", epoch)
    #torch.save(model.state_dict(), '/content/models/model_weights.pth')
    for idx, batch in enumerate(train_dataloader):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)
        labels = batch.pop("labels").to(device)

        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids)

        loss = outputs.loss

        print(f"Idx {idx}. Loss:", loss.item())
        if idx % 10 == 0:
            print(f"Idx {idx}. Loss:", loss.item())

        if idx // 100 == 0:
            torch.save(model.state_dict(), 'model_weights.pth')
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0


E:\AnacondaNew\envs\ml-course-spbu\Lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Idx 0. Loss: 10.34445858001709
Idx 0. Loss: 10.34445858001709
Idx 1. Loss: 10.260732650756836


KeyboardInterrupt: 

In [14]:
test_dataset = []
for i in range(len(X_test)):
    test_dataset.append(f'{X_test.iloc[i]["img_path"]}!****!{X_test.iloc[i]["translated"]}')
print(test_dataset[0])

dataset/train/10498937/7631593.jpg!****!Avers: the coat of arms of the USSR below the abbreviation "S.S.S.r."On the edge of the coin - a linear rim.revers: the two -line inscription "2 kopecks", below the date "1941", on the sides of wheat ears, along the edge of the coin linear rim. Gurt is sauced.


In [ ]:
import time
for example in test_dataset:

    image = example.split("!****!")[0]
    img = Image.open(image)
    display(img)
    
    
    # prepare image for the model
    inputs = processor(images=np.array(img), text="a photography of", padding="max_length", return_tensors="pt").to(device)

    generated_ids = model.generate(pixel_values=inputs.pixel_values, input_ids=inputs.input_ids)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_caption+"TTTTT"+example.split("!****!")[1])
    time.sleep(0.5)